## Variational Autoencoder: hyperparameter tuning, training and analysis.

Repetimos el workflow implementado con el AutoEncoder Convolucional, pero ahora utilizando el modelo variacional. La idea es ver cómo difiere la representación de las unidades del espacio latente.

In [ ]:
import os
import pickle
import warnings

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm

from src.data.create_data import load_data
from src.features.preprocess_data import to_dB, remove_outliers
from src.models.saving import save_configuration
from src.models.select_model import k_fold_cv
from src.models.model_wrappers import build_cae_architecture


### 1. Ajuste de hiperparámetros

Cargamos datos crudos, preprocesamos y separamos en conjuntos de entrenamiento y testeo. Luego hacemos un grid search para encontrar el mejor set de parámetros del modelo.

In [ ]:
# Cargamos los datos crudos
raw_data = load_data("raw/spm_signatures_no_noise.npy")
print(f"Shape de los datos: {raw_data.shape}")

# Removemos outliers y pasamos a dB
data_dB = to_dB(remove_outliers(raw_data, k=100))
print(f"Número de muestras luego de remover outliers: {len(data_dB)}")

In [ ]:
train_set, test_set = train_test_split(
    data_dB,
    test_size=.2,
    random_state=123
)

print(f"Tamaño de los datos de entrenamiento: {train_set.shape}")
print(f"Tamaño de los datos de testeo: {test_set.shape}")

Definimos las posibles arquitecturas junto con el resto de los hyperparametros

In [ ]:
# Fijamos la dimensión del espacio latente
LATENT_DIMENTION = 3
INPUT_SHAPE = train_set.shape[1:]

# Configuracion de las capas convolucionales
cv_layers = [
    [(4, (7, 8), 2), (4, (5, 5), 1)],
    [(4, (3, 4), 2), (4, (3, 3), 1)],
    [(4, (3, 4), 2), (4, (3, 3), 1), (4, (3, 3), 1)],
    [(4, (3, 4), 2), (4, (4, 4), 2), (4, (3, 3), 1)],
    [(4, (3, 4), 2), (8, (3, 3), 1)],
    [(4, (3, 3), 1), (4, (3, 3), 1)],
    [(4, (3, 4), 2), (4, (4, 4), 2)],
    [(4, (3, 4), 2), (16, (4, 4), 2)],
    [(4, (3, 4), 2), (8, (4, 4), 2), (16, (3, 3), 1)],
    [(4, (3, 4), 2), (4, (3, 3), 1), (4, (3, 3), 1), (4, (4, 4), 2)]
]

# Configuración de las capas densas
dense_layers = [
    (16,), (64,), (128), (256,),
    (32, 16), (16, 16), (32, 32),
    (256, 128), (256, 256)
 ]

Definida la grilla, iteramos sobre cada elemenro de la misma  y hacemos k-fold cross vaidation con k=5 para cada configuración. 

In [ ]:
# Primero chequeamos que TensorFlow detecte la gpu
tf.config.list_physical_devices('GPU')

In [ ]:
configurations_score = []
best_configuration = {'score' : 1e4}
not_tested_configuration = []

for cv_layer in tqdm(cv_layers):
    for dense_layer in dense_layers:

        # Configuración del modelo
        conv_configuration = dict(
            layers_config=cv_layer,
            kernel_initializer= 'glorot_uniform'
        )

        dense_configuration = dict(
            layers_units=dense_layer,
        )                

        configuration = {
            'conv_layers_config': conv_configuration, 
            'dense_layers_config': dense_configuration, 
            'batch_size': 16
        }

        # k-fold cross validation
        try:
            k_fold_score = k_fold_cv(
                train_set, build_cae_architecture, configuration
            )
        
        except tf.errors.ResourceExhaustedError:
            # En el caso de agotar los recursos
            not_tested_configuration.append(configuration)
            
            warnings.warn(
                ('La actual configuración agotó los recursos '
                f'de memoria y no fue evaluada: \n {configuration}'), 
                RuntimeWarning
                )

        else:
            # Actualizo los scores de las configuraciones
            configuration.update(k_fold_score)
            configurations_score.append(configuration)
                
            if configuration['score'] < best_configuration['score']:
                best_configuration = configuration 
        
        finally:
            continue     

In [ ]:
df_scores = pd.DataFrame.from_records(configurations_score).sort_values(by='score')
df_scores['conv_layers_config'] = df_scores['conv_layers_config'].apply(lambda x: x['layers_config'])
df_scores['dense_layers_config'] = df_scores['dense_layers_config'].apply(lambda x: x['layers_units'])
df_scores

Guardo los scores y la arquitectura con mejor score

In [ ]:
# Get resutls directory path
src_dir = os.path.normpath(os.getcwd() + '/..')
results_dir = os.path.join(src_dir, 'results/spm')

# File name and dir
file_name = 'vae_architectures_scores.pkl'
file_dir = os.path.join(results_dir, file_name)

# Save model_scores as pkl
with open(file_dir, 'wb') as f:
    pickle.dump(configurations_score, f)

In [ ]:
save_configuration(
    best_configuration, 
    filename='vae_architecture'
    )

Configuration saved at /home/jotavecorta/proyectos/tesis/src/models/model_architecture_spm.json


Ahora que tenemos la mejor arquitectura, probamos agregar maxpooling, dropout, earlystopping y variar el learning rate.